# Data Cleaning
### This notebook cleans and reorganzie the logs in data_collection/records.txt using spark. 
### This produces 
#### a. Two folders - userRating and userWatchLength, each containing a set of partition files about the users' ratings and users' watch history, a.k.a the time users spend on each movie <br>
#### b. Two files -  users.txt and movies.txt, containing user IDs and movie IDs that occurred in the past history.

### Create spark context and read in the records.txt

In [ ]:
import findspark
findspark.init()
import pyspark
from pyspark.sql import SQLContext
sc = pyspark.SparkContext(appName="hw")
sqlContext = SQLContext(sc)

print("Spark context started")

In [ ]:
recordsRDD = sc.textFile('records.txt')

In [ ]:
recordsRDD.count()

### Separate different types of messages

In [ ]:
watchRDD = recordsRDD.filter(lambda rec: '/data/m' in rec)
ratingRDD = recordsRDD.filter(lambda rec: '/rate/' in rec)
recommendRDD = recordsRDD.filter(lambda rec: 'recommendation' in rec)

In [ ]:
watchRDD.count()

In [ ]:
ratingRDD.count()

In [ ]:
recommendRDD.count()

### Parse users' watch history

In [ ]:
watchListRDD = watchRDD.map(lambda rec: rec.split(','))

In [ ]:
watchPairRDD = watchListRDD.map(lambda lst: (lst[1], lst[2].split('/')))

In [ ]:
userMoviePairRDD = watchPairRDD.map(lambda tup: ((tup[0], tup[1][3]), tup[1][4])).groupByKey().mapValues(list)

In [ ]:
userMoviePairRDD = userMoviePairRDD.map(lambda tup: (tup[0], tup[1][-1]))

In [ ]:
import re
userMoviePairRDD = userMoviePairRDD.mapValues(lambda v: int(re.sub('[^0-9]', '', v)))

In [ ]:
userMoviePairRDD.take(5)

In [ ]:
userMoviePairRDD.saveAsTextFile('KafkaData/userWatchLength')

### Parse users' ratings

In [ ]:
ratingListRDD = ratingRDD.map(lambda rec: rec.split(','))
ratingPairRDD = ratingListRDD.map(lambda lst: (lst[1], lst[2].split('/')))

In [ ]:
userRatingRDD = ratingPairRDD.map(lambda tup: (tup[0], tup[1][2].split('=')))

In [ ]:
userRatingPairRDD = userRatingRDD.map(lambda tup: ((tup[0], tup[1][0]), tup[1][1]))

In [ ]:
userRatingPairRDD.take(5)

In [ ]:
userRatingPairRDD.saveAsTextFile('Kafka/userRating')

### Get all users IDs and movie IDs

In [ ]:
usersRDD = recordsRDD.map(lambda rec: rec.split(',')[1]).distinct().filter(lambda s: s.isdigit())

In [ ]:
usersRDD.count()

In [ ]:
usersRDD.take(5)

In [ ]:
allUsers = sorted(usersRDD.collect())
with open('users.txt', 'w') as file:
    for user in allUsers:
        file.writelines(user + '\n')

In [ ]:
watchedMovie = userMoviePairRDD.map(lambda tup: tup[0][1]).distinct()
ratedMovie = userRatingPairRDD.map(lambda tup: tup[0][1]).distinct()

In [ ]:
allMoviesRDD = watchedMovie.union(ratedMovie).distinct()

In [ ]:
allMoviesRDD.take(5)

In [ ]:
allMovies = allMoviesRDD.collect()
with open('movies.txt', 'w') as file:
    for movie in allMovies:
        file.write(movie + '\n')

In [ ]:
allMoviesRDD.count()